In [ ]:
#######################################################################################################################
#Training the lcgNetV and lcgNetS with different values of L(layer) to colculate the Normalize Mean Square Error(NMSE)
#######################################################################################################################

In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np
import pickle 


tf.disable_eager_execution()

# Data Generation Function
def generate_data(B, K, N, snr_low, snr_high, H_org):
    x_ = np.sign(np.random.rand(B, K) - 0.5)
    y_ = np.zeros([B, N])
    w = np.random.randn(B, N)
    Hy_ = x_ * 0
    H_ = np.zeros([B, N, K])
    HH_ = np.zeros([B, K, K])
    SNR_ = np.zeros([B])

    for i in range(B):
        SNR = np.random.uniform(low=snr_low, high=snr_high)
        H = H_org
        tmp_snr= (H.T.dot(H)).trace() / K
        
        
        H_[i, :, :] = H
        y_[i, :] = H.dot(x_[i, :]) + w[i, :] * np.sqrt(tmp_snr)/np.sqrt(SNR)
        Hy_[i, :] = H.T.dot(y_[i, :])
        HH_[i, :, :] = H.T.dot(H_[i, :, :])
        SNR_[i] = SNR

    return y_, H_, Hy_, HH_, x_, SNR_,tmp_snr

# NMSE Calculation Function
def calculate_nmse(x_true, x_estimated):
    mse_per_sample = np.mean((x_true - x_estimated) ** 2, axis=1)
    power_per_sample = np.mean(x_true ** 2, axis=1)
    nmse_per_sample = mse_per_sample / power_per_sample
    avg_nmse = np.mean(nmse_per_sample)
    avg_nmse_db = 10 * np.log10(avg_nmse)
    return avg_nmse, avg_nmse_db

# Validation Function
def validate_model(sess, model, y_val, H_val, Hy_val, HH_val, x_val, sigma2_val):
    """
    Validates the model on the validation dataset and returns NMSE.
    """
    feed_dict = {
        model['y']: y_val,
        model['H']: H_val,
        model['Hy']: Hy_val,
        model['HH']: HH_val,
        model['x_true']: x_val,
        model['sigma2']: sigma2_val
    }
    nmse = sess.run(model['nmse'], feed_dict=feed_dict)
    return nmse


# Model Building Function
def build_model(N, K, L, use_vector_params):
    y = tf.placeholder(tf.float32, shape=(None, N), name='y')
    H = tf.placeholder(tf.float32, shape=(None, N, K), name='H')
    Hy = tf.placeholder(tf.float32, shape=(None, K), name='Hy')
    HH = tf.placeholder(tf.float32, shape=(None, K, K), name='HH')
    x_true = tf.placeholder(tf.float32, shape=(None, K), name='x_true')
    sigma2 = tf.placeholder(tf.float32, shape=(), name='noise_variance')  # Noise variance placeholder

    I_Nt = tf.eye(K, dtype=tf.float32)

    # Calculate A_rm and b_rm
    A_rm = HH + sigma2 * I_Nt
    b_rm = Hy

    # Initialize trainable parameters
    if use_vector_params:
        alpha = [tf.Variable(tf.zeros([K]), name=f'alpha_{t}') for t in range(L)]
        beta = [tf.Variable(tf.zeros([K]), name=f'beta_{t}') for t in range(L)]
    else:
        alpha = [tf.Variable(tf.zeros([]), name=f'alpha_{t}') for t in range(L)]
        beta = [tf.Variable(tf.zeros([]), name=f'beta_{t}') for t in range(L)]

    s_hat = tf.zeros_like(x_true, name='s_hat_init')
    r = b_rm - tf.squeeze(tf.matmul(A_rm, tf.expand_dims(s_hat, -1)), axis=-1)
    d = tf.identity(r)
    states = []

    for t in range(L):
        if use_vector_params:
            alpha_t =tf.multiply(alpha[t],d)
            beta_t = tf.multiply(beta[t],d)
        else:
            alpha_t =alpha[t] * d
            beta_t = beta[t] * d

        s_hat = s_hat + alpha_t
        r = r - tf.multiply(alpha[t], tf.squeeze(tf.matmul(A_rm, tf.expand_dims(d, -1)), axis=-1))
        d = r + beta_t
        states.append(s_hat)

    s_hat_final = states[-1]

    loss = tf.reduce_mean(tf.square(s_hat_final - x_true))
    nmse = tf.reduce_mean(tf.square(s_hat_final - x_true)) / tf.reduce_mean(tf.square(x_true))

    trinit = 0.001  # Initial learning rate
    train_ops = [tf.train.AdamOptimizer(trinit).minimize(loss, var_list=[alpha[t], beta[t]]) for t in range(L)]

    fine_tune_ops = {
    lr: tf.train.AdamOptimizer(lr).minimize(loss, var_list=[
        var for i in range(t) for var in [alpha[i], beta[i]]
    ])
    for lr in [0.001, 0.0005, 0.0001, 0.00005]
}


    return {
        'train_ops': train_ops,
        'fine_tune_ops': fine_tune_ops,
        'loss': loss,
        'nmse': nmse,
        'y': y,
        'H': H,
        'Hy': Hy,
        'HH': HH,
        'x_true': x_true,
        'sigma2': sigma2,
        's_hat_final': s_hat_final
    }

# Training with Fine-Tuning
def train_for_snr(sess, snr, model, patience=3):
    """
    Layer-wise training and fine-tuning for a specific SNR value.
    """
    print(f"Training and Fine-tuning for SNR = {snr[0]} dB...")
    L = len(model['train_ops'])
    for t in range(L):
        print(f"Training Layer {t+1}/{L} for SNR = {snr[0]} dB...")

        y_train, H_train, Hy_train, HH_train, x_train, _ ,tmp_snr= generate_data(B, K, N, snr[0], snr[1], H_org)
        y_val, H_val, Hy_val, HH_val, x_val, _,tmp_snr = generate_data(B, K, N, snr[0], snr[1], H_org)

        sigma2_val = tmp_snr/ snr[0]
        best_val_nmse = float('inf')
        patience_counter = 0

        for epoch in range(1000):
            feed_dict = {
                model['y']: y_train,
                model['H']: H_train,
                model['Hy']: Hy_train,
                model['HH']: HH_train,
                model['x_true']: x_train,
                model['sigma2']: sigma2_val
            }
            sess.run(model['train_ops'][t], feed_dict=feed_dict)

            val_nmse = validate_model(sess, model, y_val, H_val, Hy_val, HH_val, x_val, sigma2_val)
            print(f"Layer {t+1}, Epoch {epoch+1}: Validation NMSE = {val_nmse:.6f}")

            if val_nmse < best_val_nmse:
                best_val_nmse = val_nmse
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at Layer {t+1}, Epoch {epoch+1} (Best Validation NMSE = {best_val_nmse:.6f})")
                break

        print(f"Fine-tuning Layers 1 to {t+1} for SNR = {snr[0]} dB...")
        for lr, fine_tune_op in model['fine_tune_ops'].items():
            for epoch in range(200):
                feed_dict = {
                    model['y']: y_train,
                    model['H']: H_train,
                    model['Hy']: Hy_train,
                    model['HH']: HH_train,
                    model['x_true']: x_train,
                    model['sigma2']: sigma2_val
                }
                sess.run(fine_tune_op, feed_dict=feed_dict)
                val_nmse = validate_model(sess, model, y_val, H_val, Hy_val, HH_val, x_val, sigma2_val)
                print(f"Fine-tuning LR={lr}, Epoch {epoch+1}: Validation NMSE = {val_nmse:.6f}")
 
# Function to Save Parameters to File
def save_params_to_file(params, filename):
    with open(filename, 'wb') as f:
        pickle.dump(params, f)

# Function to Load Parameters from File
def load_params_from_file(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Parameters
N, K = 12, 12
B = 1000  # Reduced batch size for memory efficiency
#H_org = np.random.randn(N, K)
#frobenius_norm = np.linalg.norm(H_org, 'fro')  # Frobenius norm of the matrix
#H_org= H_org/ frobenius_norm
snr_values = [10**(20/10.0), 10**(20/10.0)]
L_list = [2,4,6,8,10]  # Number of layers to test

# File Paths for Parameters
scalar_params_file = "scalar_params.pkl"
vector_params_file = "vector_params.pkl"

# Train Scalar and Vector Models with Fine-Tuning
for L in L_list:
    print(f"\nTraining Scalar Model with L={L}")
    tf.reset_default_graph()
    scalar_model = build_model(N, K, L, use_vector_params=False)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_for_snr(sess, snr_values, scalar_model)
        
        # Save Scalar Model Parameters
        scalar_params = {var.name: sess.run(var) for var in tf.global_variables()}
        save_params_to_file(scalar_params, f"scalar_params_L{L}.pkl")
        print(f"Scalar Model Parameters for L={L} saved to scalar_params_L{L}.pkl")

    print(f"\nTraining Vector Model with L={L}")
    tf.reset_default_graph()
    vector_model = build_model(N, K, L, use_vector_params=True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_for_snr(sess, snr_values, vector_model)
        
        # Save Vector Model Parameters
        vector_params = {var.name: sess.run(var) for var in tf.global_variables()}
        save_params_to_file(vector_params, f"vector_params_L{L}.pkl")
        print(f"Vector Model Parameters for L={L} saved to vector_params_L{L}.pkl")


In [ ]:
###########################################################################################################
# The testing part where compare  the LcgNetV,LcgNetS and LMMSE base to NMSE 
########################################################################################################

In [ ]:
nmse_results = {'scalar': [], 'vector': []}
lmmse_results = []  # To store LMMSE NMSE for each L
iterations = 1  # Number of runs to average NMSE
L_list=[2,4,6,8,10]
# Initialize accumulators for averaging
scalar_nmse_accum = {L: [] for L in L_list}
vector_nmse_accum = {L: [] for L in L_list}
lmmse_nmse_accum = {L: [] for L in L_list}

for i in range(iterations):
    print(f"\nIteration {i+1}/{iterations}")

    # Generate test data once per iteration
    y_test, H_test, Hy_test, HH_test, x_test, _,tmp_snr_= generate_data(1000, K, N, snr_low=snr_values[0], snr_high=snr_values[0], H_org=H_org)

    for L in L_list:
        print(f"\nTesting Models with L={L}")

        # Compute LMMSE Estimate and NMSE in TensorFlow
        tf.reset_default_graph()
        y = tf.placeholder(tf.float32, shape=(None, N), name='y')
        H = tf.placeholder(tf.float32, shape=(None, N, K), name='H')
        x_true = tf.placeholder(tf.float32, shape=(None, K), name='x_true')
        sigma2 = tf.placeholder(tf.float32, name='sigma2')

        # LMMSE Calculation
        #tmp_snr = (H.T.dot(H)).trace() / K
        H_t = tf.transpose(H, perm=[0, 2, 1])  # Transpose of H
     
        I = tf.eye(K, batch_shape=[tf.shape(H)[0]])  # Identity matrix
        HTH = tf.matmul(H_t, H)  # H^T * H
        inv = tf.linalg.inv(HTH + sigma2 * I)  # Inverse (H^T * H + sigma^2 * I)
        HTy = tf.matmul(H_t, tf.expand_dims(y, -1))  # H^T * y
        x_lmmse = tf.squeeze(tf.matmul(inv, HTy), axis=-1)  # LMMSE estimate
       
        # LMMSE NMSE
        nmse_lmmse = tf.reduce_mean(tf.square(x_lmmse - x_true)) / tf.reduce_mean(tf.square(x_true))

        with tf.Session() as sess:
            # Feed test data to compute LMMSE NMSE
            feed_dict = {
                y: y_test,
                H: H_test,
                x_true: x_test,
                sigma2: tmp_snr_/snr_values[0]

            }
            nmse_lmmse_value = sess.run(nmse_lmmse, feed_dict=feed_dict)
            lmmse_nmse_accum[L].append(nmse_lmmse_value)  # Accumulate NMSE for LMMSE

        # Scalar Model Testing
        tf.reset_default_graph()
        scalar_model = build_model(N, K, L, use_vector_params=False)
        scalar_params_file = f"scalar_params_L{L}.pkl"
        scalar_params = load_params_from_file(scalar_params_file)  # Load scalar model parameters
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            # Restore parameters for scalar model
            for var in tf.global_variables():
                if var.name in scalar_params:
                    sess.run(var.assign(scalar_params[var.name]))

            # Scalar model feed dictionary
            feed_dict = {
                scalar_model['y']: y_test,
                scalar_model['H']: H_test,
                scalar_model['Hy']: Hy_test,
                scalar_model['HH']: HH_test, 
                scalar_model['x_true']: x_test,
                scalar_model['sigma2']: tmp_snr_/ snr_values[0]
            }

            # Compute NMSE for scalar model
            nmse_scalar = sess.run(scalar_model['nmse'], feed_dict=feed_dict)
            scalar_nmse_accum[L].append(nmse_scalar)

        # Vector Model Testing
        tf.reset_default_graph()
        vector_model = build_model(N, K, L, use_vector_params=True)
        vector_params_file = f"vector_params_L{L}.pkl"
        vector_params = load_params_from_file(vector_params_file)  # Load vector model parameters
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            # Restore parameters for vector model
            for var in tf.global_variables():
                if var.name in vector_params:
                    sess.run(var.assign(vector_params[var.name]))

            # Vector model feed dictionary
            feed_dict = {
                vector_model['y']: y_test,
                vector_model['H']: H_test,
                vector_model['Hy']: Hy_test,
                vector_model['HH']: HH_test,
                vector_model['x_true']: x_test,
                vector_model['sigma2']: tmp_snr_/ snr_values[0]
            }

            # Compute NMSE for vector model
            nmse_vector = sess.run(vector_model['nmse'], feed_dict=feed_dict)
            vector_nmse_accum[L].append(nmse_vector)

# Compute Average NMSE and Convert to dB
nmse_results_db = {'scalar': [], 'vector': []}
lmmse_results_db = []

for L in L_list:
    scalar_avg_nmse = np.mean(scalar_nmse_accum[L])
    vector_avg_nmse = np.mean(vector_nmse_accum[L])
    lmmse_avg_nmse = np.mean(lmmse_nmse_accum[L])

    nmse_results_db['scalar'].append((L, 10 * np.log10(scalar_avg_nmse)))
    nmse_results_db['vector'].append((L, 10 * np.log10(vector_avg_nmse)))
    lmmse_results_db.append((L, 10 * np.log10(lmmse_avg_nmse)))

print("\nNMSE Results (dB):")
print("Scalar Model:", nmse_results_db['scalar'])
print("Vector Model:", nmse_results_db['vector'])
print("LMMSE (dB):", lmmse_results_db)